In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to run all arguments and not just the last one
from IPython.display import Image
%matplotlib inline

In [2]:
# Sentiment analysis is a subfield of natural language processing
import pyprind
import pandas as pd
import os
# http://ai.stanford.edu/~amaas/data/sentiment/ 

In [ ]:
#basepath = '/Users/Sebastian/Desktop/aclImdb/'
basepath = 'C:/Users/309292/Desktop/Python/aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0%                          100%
[#                             ] | ETA: 00:13:48

In [ ]:
# since the class labels are sorted, we will shuffle the data frame
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('C:/Users/309292/Desktop/Python/aclImdb/movie_data.csv', index=False)

In [3]:
df = pd.read_csv('C:/Users/309292/Desktop/Python/aclImdb/movie_data.csv', sep=',', encoding='ISO-8859-1')
df.head() # to check if the exported file can be opened; there was an error due to a ' in line 164. Fixed by setting encoding

,review,sentiment
0,"Election is a Chinese mob movie, or triads in ...",1
1,I was just watching a Forensic Files marathon ...,0
2,Police Story is a stunning series of set piece...,1
3,"Dear Readers,<br /><br />The final battle betw...",1
4,I have seen The Perfect Son about three times....,1


In [4]:
# Introducing the bag of words model
# The bag of words model allows us to represent text as numerical feature vectors
# we create a vocabulary of unique tokens, eg words from the entire set of documents 
# construct a feature vector from each document that contains how often each word occurs in the particular document

In [5]:
# Transforming the words into vectors
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'
    ])
bag = count.fit_transform(docs)

In [6]:
count.vocabulary_     # the vocabulary is stored in a dictionary that maps unique words to integer indices
bag.toarray()         # the feature vectors we just created

# Each index here represents the work integer indices in the dictionary and the value signifies the count
# this is a 1 gram model. ngram_range parameter in CountVectorizer 
# to check for frequently occuring words, we use term frequency-inverse document frequency that can be used to downweight 
# frequently occuring words in the feature vector

{'and': 0,
 'is': 1,
 'one': 2,
 'shining': 3,
 'sun': 4,
 'sweet': 5,
 'the': 6,
 'two': 7,
 'weather': 8}

array([[0, 1, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1],
       [2, 3, 2, 1, 1, 1, 2, 1, 1]], dtype=int64)

In [7]:
# scikit-learn impliments a transformer called TfidfTransformer that takes the raw term frequencies from CountVectorizer and 
# transforms them into tf-idf
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()  # this applies L2 normalization by default
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.    0.56  0.56  0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.3   0.25  0.19]]


In [8]:
# Cleaning text data
df.loc[5, 'review'][-50:]

# HTML or other code in the data is not useful. Punctuation marks can be useful but the chapter dumbs it down and removes them anyway

" Best Picture. This is one of Hollywood's Classic."

In [9]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [10]:
preprocessor('is seven.<br /><br />Title (Brazil): Not Available')
preprocessor("</a>This :) is :( a test :-)!")

'is seven title brazil not available'

'this is a test :) :( :)'

In [11]:
# Applying cleaning to all reviews in the data frame
df['review'] = df['review'].apply(preprocessor)

In [12]:
# we need to split the text into individual elements.
def tokenizer(text):
    return text.split()
tokenizer("runners like running and thus, they run")

# in the context of tokenization, another useful technique is word stemming which is the process of transforming the word to its
# root that allows us to map related words to the same stem. 

['runners', 'like', 'running', 'and', 'thus,', 'they', 'run']

In [13]:
from nltk.stem.porter import PorterStemmer # PorterStemmer is the oldest and simplest stemming algorithm
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run') # words are reduced to their root form; error for thus though

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [14]:
# Stop word removal: can be used if we are not using tf-idf and working with raw or normalized term frequencies
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\309292\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [16]:
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
 if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [17]:
# Training a logistic regression for document classification
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [ ]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [19]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

3

In [ ]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

np.random.seed(0)
np.set_printoptions(precision=6)
y = [np.random.randint(3) for i in range(25)]
X = (y + np.random.randn(25)).reshape(-1, 1)

cv5_idx = list(StratifiedKFold(n_splits=5, shuffle=False, random_state=0).split(X, y))
    
cross_val_score(LogisticRegression(random_state=123), X, y, cv=cv5_idx)

In [ ]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(LogisticRegression(), {}, cv=cv5_idx, verbose=3).fit(X, y) 
gs.best_score_
cross_val_score(LogisticRegression(), X, y, cv=cv5_idx).mean()

In [18]:
# Working with bigger data - online learning and out-of-core learning
# out of core learning is a technique that a allows us to work with large datasets wihtout having supercomputer facilities

# we learnt about stochastic gradient descent which is an optimized algorithm that updates a model's weights using one sample at
# a time. we will use a partial fit function in the SGDClassifier in scikit-learn to stream the documents directly from our local 
# drive and train a logistic regression model using small minibatches of documents 

import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

def tokenizer(text): # to clean unprocessed data from the review list
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized


def stream_docs(path):    # a generator function that reads and returns one document at a time
    with open(path, 'r', encoding='ISO-8859-1') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [19]:
next(stream_docs(path="C:/Users/309292/Desktop/Python/aclImdb/movie_data.csv"))
# to check if the stream_docs function works properly

('"Election is a Chinese mob movie, or triads in this case. Every two years an election is held to decide on a new leader, and at first it seems a toss up between Big D (Tony Leung Ka Fai, or as I know him, ""The Other Tony Leung"") and Lok (Simon Yam, who was Judge in Full Contact!). Though once Lok wins, Big D refuses to accept the choice and goes to whatever lengths he can to secure recognition as the new leader. Unlike any other Asian film I watch featuring gangsters, this one is not an action movie. It has its bloody moments, when necessary, as in Goodfellas, but it\'s basically just a really effective drama. There are a lot of characters, which is really hard to keep track of, but I think that plays into the craziness of it all a bit. A 100-year-old baton, which is the symbol of power I mentioned before, changes hands several times before things settle down. And though it may appear that the film ends at the 65 or 70-minute mark, there are still a couple big surprises waiting. Si

In [26]:
# Define a function that will take a document stream from the stream_docs function and return a particular number of documents 
# specified by the size parameter
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

# we cannot use CountVectorizer for out of core learning as it requires us to hold all the data in memory to calculate
# inverse document frequencies. Another function we can use is HatchingVectorizer and is data independent and uses
# Hashing trick via the 32 bit murmurhash3 algorithm by Austin Appleby

In [27]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='C:/Users/309292/Desktop/Python/aclImdb/movie_data.csv')

In [28]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0%                          100%
[                              ]

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=1, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=1, shuffle=True, verbose=0,
       warm_start=False)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=1, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=1, shuffle=True, verbose=0,
       warm_start=False)

[#                             ] | ETA: 00:02:11

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=1, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=1, shuffle=True, verbose=0,
       warm_start=False)

...

[############################  ] | ETA: 00:00:01

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=1, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=1, shuffle=True, verbose=0,
       warm_start=False)

[############################# ] | ETA: 00:00:00

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=1, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=1, shuffle=True, verbose=0,
       warm_start=False)

[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:42


In [29]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.866


In [30]:
clf = clf.partial_fit(X_test, y_test)